In [13]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import os
import math
import itertools
from IPython.display import Image, display
import shutil
%matplotlib inline

In [14]:
input = "data2/"

In [34]:
output = "data3/"

In [16]:
mapButtons = dict()
mapButtons['NoButton'] = 1
mapButtons['Left'] = 2
mapButtons['Scroll'] = 3
mapButtons['Right'] = 4
mapButtons['Middle'] = 5
mapButtons

{'NoButton': 1, 'Left': 2, 'Scroll': 3, 'Right': 4, 'Middle': 5}

In [17]:
mapStates = dict()
mapStates['Move'] = 1
mapStates['Pressed'] = 2
mapStates['Drag'] = 3
mapStates['Released'] = 4
mapStates['Up'] = 5
mapStates['Down'] = 6
mapStates

{'Move': 1, 'Pressed': 2, 'Drag': 3, 'Released': 4, 'Up': 5, 'Down': 6}

In [18]:
# from EDA
mi_acc = -22724187369.23431
ma_acc = 92359768141.26152
acc_range = 25.3 #log(max + 1)
speed_max = 18.5

In [19]:
def normalize1(df):
    df["speed"] = df["speed"].apply(lambda x: np.log(x + 1)/speed_max)
    df["acceleration"] = df["acceleration"].apply(lambda x: np.log(np.abs(x) + 1)/acc_range)
    return df

In [25]:
def extractFeatures(df, out):
    df = df.groupby("record timestamp").last().reset_index()
    df["button"] = df["button"].apply(lambda x: mapButtons[x])
    df["state"] = df["state"].apply(lambda x: mapStates[x])
    df["time"] = df["record timestamp"].rolling(2).apply(lambda x: x[1] - x[0], raw=True)
    df["x_dist"] = df["x"].rolling(2).apply(lambda x: x[1] - x[0], raw=True)
    df["y_dist"] = df["y"].rolling(2).apply(lambda x: x[1] - x[0], raw=True)
    df["dist"] = df.apply(lambda x: np.sqrt(np.square(x["x_dist"]) + np.square(x["y_dist"])), axis=1)
    df = df.fillna(0)
    df["speed"] = df.apply(lambda x: 0 if x["time"] == 0 else x["dist"]/x["time"], axis=1)
    df["acc"] = df["speed"].rolling(2).apply(lambda x: x[1] - x[0], raw=True)
    df = df.fillna(0)
    df["acceleration"] = df.apply(lambda x: 0 if x["time"] == 0 else x["acc"]/x["time"], axis=1)
    df = df.fillna(0)
    df = df[["record timestamp", "button", "state", "x", "y", "x_dist", "y_dist", "time", "dist", "speed", "acceleration"]]
    df = normalize1(df)
    df.to_csv(out, index=False, header=True)

In [26]:
users = os.listdir(input)
for u in users:
    sessions = os.listdir(input + u)
    if not os.path.exists(output + u):
        os.makedirs(output + u)
    for sess in sessions:
        path = input + u + "/" + sess
        df = pd.read_csv(path)
        res = extractFeatures(df, output + u + "/" + sess)

In [35]:
users = os.listdir(output)
for u in users:
    sessions = os.listdir(output + u)
    for sess in sessions:
        path = output + u + "/" + sess
        df = pd.read_csv(path)
        display(df.head(10))
        break
    break

,record timestamp,button,state,x,y,x_dist,y_dist,time,dist,speed,acceleration
0,0.000,1,1,267,49,0.0,0.0,0.000,0.000000,0.000000,0.000000
1,0.341,1,1,264,49,-3.0,0.0,0.341,3.000000,0.123359,0.129976
2,0.453,1,1,195,55,-69.0,6.0,0.112,69.260378,0.347500,0.340010
3,0.565,1,1,441,109,246.0,54.0,0.112,251.857102,0.417220,0.378888
4,0.677,1,1,388,97,-53.0,-12.0,0.112,54.341513,0.334411,0.381992
5,0.784,1,1,304,78,-84.0,-19.0,0.107,86.122006,0.361726,0.316309
6,0.925,1,1,283,54,-21.0,-24.0,0.141,31.890437,0.293282,0.328855
7,1.038,1,1,266,52,-17.0,-2.0,0.113,17.117243,0.271732,0.256730
8,1.223,2,2,265,52,-1.0,0.0,0.185,1.000000,0.100386,0.263746
9,1.292,2,4,265,52,0.0,0.0,0.069,0.000000,0.000000,0.172875


# another way

In [40]:
output = "data4/"

In [41]:
min_log_acc = 23.85 #log(abs(min) + 1)
# max_log_acc = log(abs(max) + 1)
rng = 49.1 #min_log_acc + min_log_acc

def does(x):
    r = np.log(np.abs(x) + 1)
    if x >= 0:
        return np.divide(min_log_acc + r, rng)
    else:
        return np.divide(min_log_acc - r, rng)

In [42]:
def normalize2(df):
    df["speed"] = df["speed"].apply(lambda x: np.log(x + 1)/speed_max)
    df["acceleration"] = df["acceleration"].apply(lambda x: does(x))
    return df

In [43]:
def extractFeatures2(df, out):
    df = df.groupby("record timestamp").last().reset_index()
    df["button"] = df["button"].apply(lambda x: mapButtons[x])
    df["state"] = df["state"].apply(lambda x: mapStates[x])
    df["time"] = df["record timestamp"].rolling(2).apply(lambda x: x[1] - x[0], raw=True)
    df["x_dist"] = df["x"].rolling(2).apply(lambda x: x[1] - x[0], raw=True)
    df["y_dist"] = df["y"].rolling(2).apply(lambda x: x[1] - x[0], raw=True)
    df["dist"] = df.apply(lambda x: np.sqrt(np.square(x["x_dist"]) + np.square(x["y_dist"])), axis=1)
    df = df.fillna(0)
    df["speed"] = df.apply(lambda x: 0 if x["time"] == 0 else x["dist"]/x["time"], axis=1)
    df["acc"] = df["speed"].rolling(2).apply(lambda x: x[1] - x[0], raw=True)
    df = df.fillna(0)
    df["acceleration"] = df.apply(lambda x: 0 if x["time"] == 0 else x["acc"]/x["time"], axis=1)
    df = df.fillna(0)
    df = df[["record timestamp", "button", "state", "x", "y", "x_dist", "y_dist", "time", "dist", "speed", "acceleration"]]
    df = normalize2(df)
    df.to_csv(out, index=False, header=True)

In [44]:
users = os.listdir(input)
for u in users:
    sessions = os.listdir(input + u)
    if not os.path.exists(output + u):
        os.makedirs(output + u)
    for sess in sessions:
        path = input + u + "/" + sess
        df = pd.read_csv(path)
        res = extractFeatures2(df, output + u + "/" + sess)

In [45]:
users = os.listdir(output)
for u in users:
    sessions = os.listdir(output + u)
    for sess in sessions:
        path = output + u + "/" + sess
        df = pd.read_csv(path)
        display(df.head(10))
        break
    break

,record timestamp,button,state,x,y,x_dist,y_dist,time,dist,speed,acceleration
0,0.000,1,1,267,49,0.0,0.0,0.000,0.000000,0.000000,0.485743
1,0.341,1,1,264,49,-3.0,0.0,0.341,3.000000,0.123359,0.552717
2,0.453,1,1,195,55,-69.0,6.0,0.112,69.260378,0.347500,0.660942
3,0.565,1,1,441,109,246.0,54.0,0.112,251.857102,0.417220,0.680975
4,0.677,1,1,388,97,-53.0,-12.0,0.112,54.341513,0.334411,0.288913
5,0.784,1,1,304,78,-84.0,-19.0,0.107,86.122006,0.361726,0.648729
6,0.925,1,1,283,54,-21.0,-24.0,0.141,31.890437,0.293282,0.316293
7,1.038,1,1,266,52,-17.0,-2.0,0.113,17.117243,0.271732,0.353457
8,1.223,2,2,265,52,-1.0,0.0,0.185,1.000000,0.100386,0.349841
9,1.292,2,4,265,52,0.0,0.0,0.069,0.000000,0.000000,0.396665


# check diff between output from both ways

In [46]:
data = "/user21/session_1065465945"

In [47]:
for d in ["data3", "data4"]:
    df = pd.read_csv(d + data)
    display(df.head(10))

,record timestamp,button,state,x,y,x_dist,y_dist,time,dist,speed,acceleration
0,0.000,1,1,267,49,0.0,0.0,0.000,0.000000,0.000000,0.000000
1,0.341,1,1,264,49,-3.0,0.0,0.341,3.000000,0.123359,0.129976
2,0.453,1,1,195,55,-69.0,6.0,0.112,69.260378,0.347500,0.340010
3,0.565,1,1,441,109,246.0,54.0,0.112,251.857102,0.417220,0.378888
4,0.677,1,1,388,97,-53.0,-12.0,0.112,54.341513,0.334411,0.381992
5,0.784,1,1,304,78,-84.0,-19.0,0.107,86.122006,0.361726,0.316309
6,0.925,1,1,283,54,-21.0,-24.0,0.141,31.890437,0.293282,0.328855
7,1.038,1,1,266,52,-17.0,-2.0,0.113,17.117243,0.271732,0.256730
8,1.223,2,2,265,52,-1.0,0.0,0.185,1.000000,0.100386,0.263746
9,1.292,2,4,265,52,0.0,0.0,0.069,0.000000,0.000000,0.172875


,record timestamp,button,state,x,y,x_dist,y_dist,time,dist,speed,acceleration
0,0.000,1,1,267,49,0.0,0.0,0.000,0.000000,0.000000,0.485743
1,0.341,1,1,264,49,-3.0,0.0,0.341,3.000000,0.123359,0.552717
2,0.453,1,1,195,55,-69.0,6.0,0.112,69.260378,0.347500,0.660942
3,0.565,1,1,441,109,246.0,54.0,0.112,251.857102,0.417220,0.680975
4,0.677,1,1,388,97,-53.0,-12.0,0.112,54.341513,0.334411,0.288913
5,0.784,1,1,304,78,-84.0,-19.0,0.107,86.122006,0.361726,0.648729
6,0.925,1,1,283,54,-21.0,-24.0,0.141,31.890437,0.293282,0.316293
7,1.038,1,1,266,52,-17.0,-2.0,0.113,17.117243,0.271732,0.353457
8,1.223,2,2,265,52,-1.0,0.0,0.185,1.000000,0.100386,0.349841
9,1.292,2,4,265,52,0.0,0.0,0.069,0.000000,0.000000,0.396665
